In [1]:
import pandas as pd
import pyarrow.dataset as pads
import pyarrow as pa
import os
import duckdb as db

pd.set_option('display.max_columns', None)

In [2]:
# Point to where smt_2025.db was created
con = db.connect("/Users/buttz/Desktop/SMTChallenge2025/smt_2025.db")

In [3]:
event_code_dict = {
    1: 'pitch',
    2: 'ball acquired',
    3: 'throw (ball-in-play)',
    4: 'ball hit into play',
    5: 'end of play',
    6: 'pickoff throw',
    7: 'ball acquired - unknown field position',
    8: 'throw (ball-in-play) - unknown field position',
    9: 'ball deflection',
    10: 'ball deflection off of wall',
    11: 'home run',
    16: 'ball bounce'
}

player_position_dict = {
    1: 'pitcher',
    2: 'catcher',
    3: 'first basemen',
    4: 'second basemen',
    5: 'third basemen',
    6: 'shortstop',
    7: 'left field',
    8: 'center field',
    9: 'right field',
    10: 'batter',
    11: 'runner on first base',
    12: 'runner on second base',
    13: 'runner on third base',
    255: 'ball event with no player (e.g., ball bounce)',
    14: 'home plate umpire',
    15: 'field umpire #1',
    16: 'field umpire #2',
    17: 'field umpire #3',
    18: 'first base coach',
    19: 'third base coach'
}

In [30]:
con.execute("SELECT * FROM player_pos WHERE player_position IN (2) LIMIT 1_000_000").df().copy()[['field_x', 'field_y']].mean()

field_x    0.684056
field_y   -4.644170
dtype: float64

In [4]:
# This query takes awhile just because there's so many rows for each timestamp and there are lots of timestamps ~18 million rows
player_pos = con.execute("SELECT * FROM player_pos WHERE player_position IN (7, 8, 9)").df().copy()
display(player_pos.shape)
player_pos

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(18132330, 10)

,game_str,play_id,timestamp,player_position,field_x,field_y,home_team,away_team,year,day
0,y1_d069_ACN_QEA,1,14853,7,-142.9887,249.6216,QEA,ACN,year_1,day_069
1,y1_d069_ACN_QEA,1,14853,8,-6.7491,313.4958,QEA,ACN,year_1,day_069
2,y1_d069_ACN_QEA,1,14853,9,122.9529,261.3693,QEA,ACN,year_1,day_069
3,y1_d069_ACN_QEA,1,14903,7,-142.9800,249.6138,QEA,ACN,year_1,day_069
4,y1_d069_ACN_QEA,1,14903,8,-6.7491,313.4958,QEA,ACN,year_1,day_069
...,...,...,...,...,...,...,...,...,...,...
18132325,y1_d058_WZR_YJD,330,12032975,8,-13.1577,216.3279,YJD,WZR,year_1,day_058
18132326,y1_d058_WZR_YJD,330,12032975,9,94.3239,194.6700,YJD,WZR,year_1,day_058
18132327,y1_d058_WZR_YJD,330,12033008,7,-65.5203,204.8616,YJD,WZR,year_1,day_058
18132328,y1_d058_WZR_YJD,330,12033008,8,-13.1823,216.4344,YJD,WZR,year_1,day_058


In [ ]:
def player_pos_df_changer(df: pd.DataFrame) -> pd.DataFrame:
    """
    Decreases the amount of rows for player_pos by moving each player's position as a column instead of its own row for each timestamp.
    Makes it easier to work with player_pos --> easier to merge with others and see how each player (at each position) is moving at each timestamp for any given play.

    Arguments: Dataframe
    """
    df = df.copy()
    df = df.pivot(index=['game_str', 'play_id', 'timestamp'], columns='player_position').copy()
    df.columns = [f'{i[0]}_{i[1]}' for i in df.columns]
    df.columns = [i if i.startswith('field_x') or i.startswith('field_y') else i.rsplit('_', 1)[0] for i in df.columns]
    df = df.loc[:, ~df.columns.duplicated()].copy()
    df = df[['field_x_7', 'field_y_7',
             'field_x_8', 'field_y_8',
             'field_x_9', 'field_y_9']].copy()
    return df.reset_index()

In [16]:
# Gets it to around only 6 million rows (1/3 of the original player_pos df) --> takes around 5-6 min to load player_pos + transform it
player_pos_transformed = player_pos_df_changer(player_pos).copy()
player_pos_transformed

,game_str,play_id,timestamp,field_x_7,field_y_7,field_x_8,field_y_8,field_x_9,field_y_9
0,y1_d001_CGA_QEA,1,8699,-138.6321,230.9421,-31.1307,307.0320,109.1400,258.7404
1,y1_d001_CGA_QEA,1,8749,-138.5343,230.9007,-31.1985,306.8640,109.1367,258.6951
2,y1_d001_CGA_QEA,1,8799,-138.4380,230.8596,-31.2663,306.6960,109.1358,258.6498
3,y1_d001_CGA_QEA,1,8849,-138.3426,230.8182,-31.3341,306.5250,109.1370,258.6045
4,y1_d001_CGA_QEA,1,8899,-138.2490,230.7768,-31.4019,306.3570,109.1400,258.5595
...,...,...,...,...,...,...,...,...,...
6051002,y2_d099_YJD_RZQ,292,10147948,-111.0312,268.1436,-15.5784,308.7033,118.3521,257.0847
6051003,y2_d099_YJD_RZQ,292,10147998,-111.0039,268.1094,-15.6582,308.6394,118.3509,257.0739
6051004,y2_d099_YJD_RZQ,292,10148048,-110.9769,268.0752,-15.7377,308.5755,118.3494,257.0628
6051005,y2_d099_YJD_RZQ,292,10148098,-110.9499,268.0416,-15.8169,308.5116,118.3482,257.0520


In [7]:
# Query a table
rosters = con.execute("SELECT * FROM rosters").df().copy()
display(rosters.shape)
rosters.head()

(143, 5)

,DYE,OXG,QEA,RZQ,YJD
0,DYE-0009,OXG-0018,QEA-0008,RZQ-0003,YJD-0001
1,DYE-0012,OXG-0029,QEA-0010,RZQ-0004,YJD-0002
2,DYE-0015,OXG-0052,QEA-0013,RZQ-0005,YJD-0007
3,DYE-0022,OXG-0054,QEA-0027,RZQ-0006,YJD-0011
4,DYE-0023,OXG-0061,QEA-0039,RZQ-0014,YJD-0016


In [9]:
game_events = con.execute("SELECT * FROM game_events").df().copy()
game_events.loc[:, 'event_code_name'] = game_events['event_code'].map(event_code_dict)
game_events.loc[:, 'player_position_name'] = game_events['player_position'].map(player_position_dict)
display(game_events.shape)
game_events

(301314, 13)

,game_str,play_id,at_bat,play_per_game,timestamp,player_position,event_code,home_team,away_team,year,day,event_code_name,player_position_name
0,y1_d069_ACN_QEA,1,1,1,14853,1,1,QEA,ACN,year_1,day_069,pitch,pitcher
1,y1_d069_ACN_QEA,1,1,1,15303,2,2,QEA,ACN,year_1,day_069,ball acquired,catcher
2,y1_d069_ACN_QEA,1,1,1,15303,0,5,QEA,ACN,year_1,day_069,end of play,NaN
3,y1_d069_ACN_QEA,2,1,2,27753,1,1,QEA,ACN,year_1,day_069,pitch,pitcher
4,y1_d069_ACN_QEA,2,1,2,28253,2,2,QEA,ACN,year_1,day_069,ball acquired,catcher
...,...,...,...,...,...,...,...,...,...,...,...,...,...
301309,y1_d058_WZR_YJD,330,93,330,12027662,255,16,YJD,WZR,year_1,day_058,ball bounce,"ball event with no player (e.g., ball bounce)"
301310,y1_d058_WZR_YJD,330,93,330,12027959,255,16,YJD,WZR,year_1,day_058,ball bounce,"ball event with no player (e.g., ball bounce)"
301311,y1_d058_WZR_YJD,330,93,330,12029972,8,2,YJD,WZR,year_1,day_058,ball acquired,center field
301312,y1_d058_WZR_YJD,330,93,330,12031028,8,3,YJD,WZR,year_1,day_058,throw (ball-in-play),center field


In [17]:
game_events.merge(player_pos_transformed, on=['game_str', 'timestamp','play_id'], how='left')

,game_str,play_id,at_bat,play_per_game,timestamp,player_position,event_code,home_team,away_team,year,day,event_code_name,player_position_name,field_x_7,field_y_7,field_x_8,field_y_8,field_x_9,field_y_9
0,y1_d069_ACN_QEA,1,1,1,14853,1,1,QEA,ACN,year_1,day_069,pitch,pitcher,-142.9887,249.6216,-6.7491,313.4958,122.9529,261.3693
1,y1_d069_ACN_QEA,1,1,1,15303,2,2,QEA,ACN,year_1,day_069,ball acquired,catcher,-142.9101,249.5517,-6.7491,313.4958,122.8617,261.3048
2,y1_d069_ACN_QEA,1,1,1,15303,0,5,QEA,ACN,year_1,day_069,end of play,NaN,-142.9101,249.5517,-6.7491,313.4958,122.8617,261.3048
3,y1_d069_ACN_QEA,2,1,2,27753,1,1,QEA,ACN,year_1,day_069,pitch,pitcher,-142.8819,251.8848,-11.2248,312.3438,123.5310,260.6589
4,y1_d069_ACN_QEA,2,1,2,28253,2,2,QEA,ACN,year_1,day_069,ball acquired,catcher,-142.9683,251.9652,-11.1687,312.3180,123.5310,260.6589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301309,y1_d058_WZR_YJD,330,93,330,12027662,255,16,YJD,WZR,year_1,day_058,ball bounce,"ball event with no player (e.g., ball bounce)",-118.1913,238.2201,-5.1789,278.3676,116.9985,237.7755
301310,y1_d058_WZR_YJD,330,93,330,12027959,255,16,YJD,WZR,year_1,day_058,ball bounce,"ball event with no player (e.g., ball bounce)",-114.4884,236.1858,-3.6501,272.7831,114.6690,236.4663
301311,y1_d058_WZR_YJD,330,93,330,12029972,8,2,YJD,WZR,year_1,day_058,ball acquired,center field,-89.9115,228.5664,-8.3373,236.2626,104.2200,221.3469
301312,y1_d058_WZR_YJD,330,93,330,12031028,8,3,YJD,WZR,year_1,day_058,throw (ball-in-play),center field,-79.7016,221.7468,-9.6381,220.0131,101.1591,211.8525


In [19]:
ball_pos = con.execute("SELECT * FROM ball_pos").df().copy()
display(ball_pos.shape)
ball_pos

(2303284, 10)

,game_str,play_id,timestamp,ball_position_x,ball_position_y,ball_position_z,home_team,away_team,year,day
0,y1_d069_ACN_QEA,1,14853,3.500460,52.04430,5.714190,QEA,ACN,year_1,day_069
1,y1_d069_ACN_QEA,1,14903,3.210720,45.69720,5.543220,QEA,ACN,year_1,day_069
2,y1_d069_ACN_QEA,1,14953,2.929950,39.41790,5.319090,QEA,ACN,year_1,day_069
3,y1_d069_ACN_QEA,1,15003,2.658168,33.20670,5.041800,QEA,ACN,year_1,day_069
4,y1_d069_ACN_QEA,1,15053,2.395365,27.06366,4.711320,QEA,ACN,year_1,day_069
...,...,...,...,...,...,...,...,...,...,...
2303279,y1_d058_WZR_YJD,330,12032876,-4.401720,22.31895,5.104110,YJD,WZR,year_1,day_058
2303280,y1_d058_WZR_YJD,330,12032909,-4.344270,19.41483,4.336530,YJD,WZR,year_1,day_058
2303281,y1_d058_WZR_YJD,330,12032942,-4.287780,16.53150,3.542400,YJD,WZR,year_1,day_058
2303282,y1_d058_WZR_YJD,330,12032975,-4.232250,13.66896,2.721744,YJD,WZR,year_1,day_058


In [21]:
game_info = con.execute("SELECT * FROM game_info").df().copy()
display(game_info.shape)
game_info

(72566, 21)

,game_str,home_team,away_team,at_bat,play_per_game,top_bottom_inning,pitcher,catcher,first_base,second_base,third_base,shortstop,left_field,center_field,right_field,batter,first_baserunner,second_baserunner,third_baserunner,year,day
0,y1_d069_ACN_QEA,QEA,ACN,1.0,1.0,top,QEA-0410,QEA-0071,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0364,QEA-0365,QEA-0249,ACN-2455,NA,NA,NA,year_1,day_069
1,y1_d069_ACN_QEA,QEA,ACN,1.0,2.0,top,QEA-0410,QEA-0071,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0364,QEA-0365,QEA-0249,ACN-2455,NA,NA,NA,year_1,day_069
2,y1_d069_ACN_QEA,QEA,ACN,1.0,3.0,top,QEA-0410,QEA-0071,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0364,QEA-0365,QEA-0249,ACN-2455,NA,NA,NA,year_1,day_069
3,y1_d069_ACN_QEA,QEA,ACN,2.0,4.0,top,QEA-0410,QEA-0071,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0364,QEA-0365,QEA-0249,ACN-1416,NA,NA,NA,year_1,day_069
4,y1_d069_ACN_QEA,QEA,ACN,69.0,134.0,top,QEA-0410,QEA-0071,QEA-0263,QEA-0277,QEA-0232,QEA-0027,QEA-0364,QEA-0365,QEA-0249,ACN-1147,NA,NA,NA,year_1,day_069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72561,y1_d058_WZR_YJD,YJD,WZR,91.0,325.0,top,YJD-0157,YJD-0007,YJD-0373,YJD-0060,YJD-0284,YJD-0172,YJD-0398,YJD-0185,YJD-0340,WZR-1285,NA,NA,NA,year_1,day_058
72562,y1_d058_WZR_YJD,YJD,WZR,93.0,326.0,top,YJD-0157,YJD-0007,YJD-0373,YJD-0060,YJD-0284,YJD-0172,YJD-0398,YJD-0185,YJD-0340,WZR-1285,NA,NA,NA,year_1,day_058
72563,y1_d058_WZR_YJD,YJD,WZR,93.0,327.0,top,YJD-0157,YJD-0007,YJD-0373,YJD-0060,YJD-0284,YJD-0172,YJD-0398,YJD-0185,YJD-0340,WZR-1285,NA,NA,NA,year_1,day_058
72564,y1_d058_WZR_YJD,YJD,WZR,93.0,329.0,top,YJD-0157,YJD-0007,YJD-0373,YJD-0060,YJD-0284,YJD-0172,YJD-0398,YJD-0185,YJD-0340,WZR-1942,WZR-1285,NA,NA,year_1,day_058


In [24]:
def info_table(df: pd.DataFrame) -> pd.DataFrame:
    """
    Displays information about the dataframe with # of missing values, unique numbers, datatypes, etc.
    
    Arguments: Dataframe
    """
    df = df.copy()
    dtype = df.dtypes
    num_unique = df.T.apply(lambda x: x.nunique(), axis=1)
    nan_total = df.isnull().sum()
    nan_percent = (df.isnull().sum()/df.isnull().count()*100)
    quality_df = pd.concat([nan_total, nan_percent, num_unique, dtype], axis=1, keys=['total_NaNs', 'NaN%', 'num_unique', 'dtypes'])
    display(quality_df)

In [25]:
info_table(game_events.merge(player_pos_transformed, on=['game_str', 'timestamp','play_id'], how='left'))

,total_NaNs,NaN%,num_unique,dtypes
game_str,0,0.000000,274,object
play_id,0,0.000000,441,int64
at_bat,0,0.000000,110,object
play_per_game,0,0.000000,441,int64
timestamp,0,0.000000,247612,int64
player_position,0,0.000000,15,int64
event_code,0,0.000000,12,int64
home_team,0,0.000000,3,object
away_team,0,0.000000,74,object
year,0,0.000000,2,object


In [26]:
game_events.merge(player_pos_transformed, on=['game_str', 'timestamp','play_id'], how='left')[game_events.merge(player_pos_transformed, on=['game_str', 'timestamp','play_id'], how='left')['field_x_7'].isna()]

,game_str,play_id,at_bat,play_per_game,timestamp,player_position,event_code,home_team,away_team,year,day,event_code_name,player_position_name,field_x_7,field_y_7,field_x_8,field_y_8,field_x_9,field_y_9
3680,y1_d072_ACN_QEA,150,2,9,167399,0,5,QEA,ACN,year_1,day_072,end of play,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8107,y1_d055_APZ_QEA,60,14,60,1868482,1,1,QEA,APZ,year_1,day_055,pitch,pitcher,NaN,NaN,-23.0415,304.0782,NaN,NaN
12119,y1_d004_CGA_QEA,16,4,16,400933,1,1,QEA,CGA,year_1,day_004,pitch,pitcher,NaN,NaN,NaN,NaN,NaN,NaN
12120,y1_d004_CGA_QEA,16,4,16,401395,2,2,QEA,CGA,year_1,day_004,ball acquired,catcher,NaN,NaN,NaN,NaN,NaN,NaN
12693,y1_d004_CGA_QEA,220,55,220,8247085,1,1,QEA,CGA,year_1,day_004,pitch,pitcher,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264212,y2_d054_PTB_YJD,139,33,139,4092030,1,1,YJD,PTB,year_2,day_054,pitch,pitcher,NaN,NaN,22.4841,316.1463,143.9670,249.0231
270487,y2_d043_RZQ_YJD,304,74,304,9710151,1,1,YJD,RZQ,year_2,day_043,pitch,pitcher,NaN,NaN,NaN,NaN,NaN,NaN
270488,y2_d043_RZQ_YJD,304,74,304,9710601,10,4,YJD,RZQ,year_2,day_043,ball hit into play,batter,NaN,NaN,NaN,NaN,NaN,NaN
270489,y2_d043_RZQ_YJD,304,74,304,9710851,0,5,YJD,RZQ,year_2,day_043,end of play,NaN,NaN,NaN,NaN,NaN,NaN,NaN
